# 정적크롤링 과제(이준목)

- 알라딘 주간베스트 page3까지 (이미지, 제목, 작가, 가격)크롤링 후 image 저장 및 DB에(제목, 작가, 가격) 저장

In [39]:
from bs4 import BeautifulSoup as bs
import requests
import re
# import json
import urllib.request as req
import pandas as pd
import pymysql

base = "https://www.aladin.co.kr/shop/common/wbest.aspx?BestType=Bestseller&BranchType=1&CID=0&page="


In [16]:
# 이미지 가져와서 저장하는 함수
def save_img(src, title):
    pattern = r'[가-힣a-zA-Z0-9]+'
    result = re.findall(pattern, title)
    result = ' '.join(result)

    path = f'data/images/{result}.jpg'

    req.urlretrieve(src, path)
    print(path, '저장됨')

In [ ]:
books = []
for i in range(1,3):
    url = base+str(i)
    res = requests.get(url)
    soup = bs(res.text, 'html.parser')

    item_list = soup.select("#Myform > div.ss_book_box")
    #print(item_list)

    for item in item_list:
    # 제목
        title = item.select_one('a.bo3').text
        #print(title)
        # #Myform > div:nth-child(3) > table > tbody > tr > td:nth-child(3) > table > tbody > tr:nth-child(1) > td:nth-child(1) > div:nth-child(1) > ul > li:nth-child(3) > a:nth-child(1)
    # 작가명
        author = item.select_one('ul > li:nth-child(3) > a:nth-child(1)')
        if author:
            author = author.text
        else:
            author = item.select_one('ul > li:nth-child(2) > a:nth-child(1)').text
        #print(author)
    # 가격
        #Myform > div:nth-child(31) > table > tbody > tr > td:nth-child(3) > table > tbody > tr:nth-child(1) > td:nth-child(1) > div:nth-child(1) > ul > li:nth-child(4) > span:nth-child(1)
        price = item.select_one('ul > li:nth-child(4) > span:nth-child(1)')
        if price:
            price = price.text.replace(',','')
        else:
            price = item.select_one('ul > li:nth-child(3) > span:nth-child(1)').text.replace(',','')
        #print(price)
    # 이미지 출력
        #Myform > div:nth-child(32) > table > tbody > tr > td:nth-child(2) > table > tbody > tr:nth-child(1) > td > div.cover_area > a > div > div > img.front_cover
        img = item.select_one('div.cover_area > a > div > div > img.front_cover')
        if img :
            src = img['src']
        else:
            #Myform > div:nth-child(31) > table > tbody > tr > td:nth-child(2) > table > tbody > tr:nth-child(1) > td > div > a > img
            src = item.select_one('td > div > a > img.i_cover')['src']
        save_img(src, title)
        
    # 자료 DB에 추가    
        data = (title, author, int(price))
        books.append(data)
    # print(len(books))


data/images/소년이 온다.jpg 저장됨
data/images/스토너.jpg 저장됨
data/images/그릿 Grit.jpg 저장됨
data/images/봄밤의 모든 것.jpg 저장됨
data/images/양심.jpg 저장됨
data/images/초역 부처의 말.jpg 저장됨
data/images/듀얼 브레인.jpg 저장됨
data/images/모순.jpg 저장됨
data/images/2025 큰별쌤 최태성의 별 별한국사 한국사능력검정시험 심화 1 2 3급 상.jpg 저장됨
data/images/ETS 토익 정기시험 기출문제집 1000 Vol 4 Reading 리딩.jpg 저장됨
data/images/황현필의 진보를 위한 역사.jpg 저장됨
data/images/ETS 토익 정기시험 기출문제집 1000 Vol 4 Listening 리스닝.jpg 저장됨
data/images/해커스 토익 기출 VOCA 보카.jpg 저장됨
data/images/삶이라는 완벽한 농담.jpg 저장됨
data/images/2025 큰별쌤 최태성의 별 별한국사 한국사능력검정시험 심화 1 2 3급 하.jpg 저장됨
data/images/채식주의자 리마스터판.jpg 저장됨
data/images/어른의 행복은 조용하다.jpg 저장됨
data/images/작별하지 않는다.jpg 저장됨
data/images/자살의 연구.jpg 저장됨
data/images/마음의 기술.jpg 저장됨
data/images/침묵의 퍼레이드.jpg 저장됨
data/images/급류.jpg 저장됨
data/images/학생회에도 구멍은 있다 8.jpg 저장됨
data/images/원페이지 교육학 기본서 상.jpg 저장됨
data/images/맨큐의 경제학.jpg 저장됨
data/images/환율의 대전환.jpg 저장됨
data/images/트럼프 시대의 지정학과 비트코인.jpg 저장됨
data/images/슈퍼 뒤에서 담배 피우는 두 사람 5 특장판.jpg 저장됨
data/images/학생회에도 구멍은 있다 공식

In [ ]:
# 데이터프레임화
df = pd.DataFrame(books, columns=['title', 'author', 'price'])
df.to_csv('data/aladinBooks2.csv', index=False)
df

,title,author,price
0,소년이 온다,한강,15000
1,스토너,존 윌리엄스,16800
2,그릿 Grit,김주환,18800
3,봄밤의 모든 것,백수린,17000
4,양심,최재천,18000
...,...,...,...
95,패턴과 구조 영문법,윤규철,19000
96,Brown 일반화학 (일반화학교재연구회),Theodore L. Brown,55000
97,히카루가 죽은 여름 5,모쿠모쿠렌,6500
98,대온실 수리 보고서,김금희,18000


In [ ]:
# DB에 저장
conn = pymysql.connect(
    host='localhost',
    user='root',
    password='1234',
    db='pydb',
    charset='utf8' 
    )
print(conn)
cursor=conn.cursor()
sql="insert into tblbook2(title, author, price) values(%s,%s,%s)"
cursor.executemany(sql, books)
conn.commit()
conn.close()